**TP3 - Web Scraping**

**Aluno Giovano Panatta**

# Parte 1 XPath - Acesse o arquivo XML em anexo

Exercício 1:

Utilize XPath absoluto com a biblioteca lxml para selecionar:

O nome do diretor do segundo filme listado;
O id do penúltimo filme listado, independentemente da quantidade de filmes na lista. 

In [1]:
pip install --q lxml


^C
Note: you may need to restart the kernel to use updated packages.


Estrutura Geral:

O arquivo XML possui uma tag raiz chamada <library>, que contém duas seções principais: <books> e <movies>. Dentro de cada uma dessas seções, há uma lista de itens (<book> ou <movie>) que contêm informações sobre livros e filmes, respectivamente.

1. Seção de Livros (<books>):
Esta seção contém uma lista de livros, representados pela tag <book>.
Cada <book> tem dois atributos: id (um identificador único para o livro) e category (a categoria do livro, como "fantasy" ou "science_fiction").
Cada <book> contém as seguintes informações:
Título (<title>) – O nome do livro.
Autor (<author>) – O nome do autor.
Ano (<year>) – O ano de publicação.
Preço (<price>) – O preço do livro, com o atributo currency indicando a moeda (USD, NZD, GBP).

In [8]:
# Importamos a função etree da biblioteca lxml, que nos permite trabalhar com XML e fazer consultas XPath
from lxml import etree

# Carregamos o arquivo XML (caminho absoluto)
tree = etree.parse("C:\infnet_ultimo_semestre\TP3_web_scraping\TP3.xml")

# XPath 1: Selecionando o nome do diretor do segundo filme
# O caminho absoluto até o nome do diretor é /library/movies/movie[2]/director
director_second_movie = tree.xpath('/library/movies/movie[2]/director/text()')[0]

# XPath 2: Selecionando o ID do penúltimo filme
# Usamos last()-1 para pegar o penúltimo item
penultimate_movie_id = tree.xpath('/library/movies/movie[last()-1]/@id')[0]

# Exibindo os resultados
print(f"Diretor do segundo filme: {director_second_movie}")
print(f"ID do penúltimo filme: {penultimate_movie_id}")


Diretor do segundo filme: Francis Ford Coppola
ID do penúltimo filme: 102


# Exercício 2:

Utilize XPath relativo com a biblioteca lxml para selecionar:

Os títulos dos livros ou filmes da categoria fantasia;

Os nomes (siglas) das moedas utilizadas nos preços que não são dólares americanos (USD);

Os preços com valor numérico maior que 15 (independentemente da moeda);

Todos os títulos que começam com "The".

In [9]:
from lxml import etree

# Carregando o arquivo XML (substitua com o caminho correto do arquivo)
tree = etree.parse("C:\infnet_ultimo_semestre\TP3_web_scraping\TP3.xml")


# Usamos XPath relativo para pegar títulos de livros ou filmes com category="fantasy"
titles_fantasy = tree.xpath('//book[@category="fantasy"]/title/text() | //movie[@category="fantasy"]/title/text()')


# Usamos XPath para selecionar o atributo currency de todos os elementos <price> que não são USD
non_usd_currencies = tree.xpath('//price[@currency!="USD"]/@currency')


# Utilizamos XPath para pegar o valor de texto dos elementos <price> cujo valor é maior que 15
prices_above_15 = tree.xpath('//price[number(text()) > 15]/text()')


# Utilizamos a função XPath starts-with() para pegar títulos que começam com "The"
titles_starting_with_the = tree.xpath('//title[starts-with(text(), "The")]/text()')

# Exibindo os resultados
print(f"Títulos da categoria fantasia: {titles_fantasy}")
print(f"Moedas que não são USD: {non_usd_currencies}")
print(f"Preços maiores que 15: {prices_above_15}")
print(f"Títulos que começam com 'The': {titles_starting_with_the}")


Títulos da categoria fantasia: ["Harry Potter and the Philosopher's Stone", 'The Hobbit']
Moedas que não são USD: ['NZD', 'GBP', 'BRL']
Preços maiores que 15: ['19.99', '15.99', '19.99']
Títulos que começam com 'The': ['The Hobbit', 'The Godfather']


# Explicação sobre XPath Relativo com lxml

**XPath Relativo** é uma técnica utilizada para navegar e extrair dados de documentos XML ou HTML sem precisar especificar o caminho completo desde a raiz. Ele é eficiente para selecionar elementos com base em atributos ou conteúdo, independentemente de sua posição exata na hierarquia do documento.

### Vantagens:
- **Flexibilidade**: Permite selecionar elementos de qualquer lugar no documento, sem a necessidade de um caminho absoluto.
- **Seleção específica**: Usamos expressões como `@atributo`, `text()`, e funções como `starts-with()` para buscar elementos com precisão.

### Exemplos práticos:
- **`//book[@category="fantasy"]`**: Seleciona todos os livros da categoria "fantasia".
- **`//price[@currency!="USD"]/@currency`**: Seleciona todas as moedas que não são USD.
- **`//price[number(text()) > 15]`**: Seleciona preços maiores que 15.
- **`//title[starts-with(text(), "The")]`**: Seleciona todos os títulos que começam com "The".

Essa técnica é ideal para extrações específicas e eficientes, sem a necessidade de conhecer a estrutura exata do documento.


# Parte 2 Acesse o arquivo HTML em anexo

# Exercício 3:

Utilize (apenas) seletores de CSS com lxml para selecionar:

Todos os nomes de países dos atletas;
Todos os nomes de atletas que possuem a letra J (maiúscula);
Todos os nomes de atletas dos EUA (USA) e da Rússia.

In [10]:
pip install lxml cssselect


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


O arquivo HTML fornecido representa um "Dashboard de Desempenho de Atletas". Ele contém uma lista de cinco atletas, cada um com informações detalhadas sobre país, idade, desempenho em competições, e resultados de testes físicos. A estrutura segue um padrão de repetição para cada atleta, sendo que cada um é identificado por uma div com a classe "athlete" e um id único (athlete-1, athlete-2, etc.).




In [11]:
# Cada atleta tem as seguintes informações:
# 
# Nome: Está na tag h2 com a classe "name".
# País: Está no parágrafo <p> com a classe "country".
# Idade: Está no parágrafo <p> com a classe "age".
# Desempenho: Está no parágrafo <p> com a classe "performance", indicando a posição no ranking.
# Resultados dos Testes: Div com a classe "tests", contendo resultados de dois tipos de testes: velocidade e resistência.
#
# Essas informações estão organizadas em blocos repetidos para cada atleta. Vamos usar essa estrutura para selecionar dados utilizando seletores CSS.


In [12]:
from lxml import etree
from lxml.cssselect import CSSSelector

# Usando HTMLParser para processar HTML
parser = etree.HTMLParser()

# Carregando o arquivo HTML com o parser adequado
tree = etree.parse(r"C:\infnet_ultimo_semestre\TP3_web_scraping\TP3.html", parser)


# Selecionando todos os elementos <p> com a classe "country"
countries_selector = CSSSelector('.country')
countries = [country.text for country in countries_selector(tree)]


# Selecionando todos os elementos <h2> com a classe "name" e filtrando os que têm "J"
names_selector = CSSSelector('.name')
names_with_j = [name.text for name in names_selector(tree) if 'J' in name.text]

# Selecionando os atletas e filtrando pelos países "USA" e "Russia"
athletes_from_usa_russia = [
    name.text for name, country in zip(names_selector(tree), countries_selector(tree)) 
    if country.text in ['USA', 'Russia']
]

# Exibindo os resultados
print(f"Países dos atletas: {countries}")
print(f"Nomes de atletas com 'J': {names_with_j}")
print(f"Atletas dos EUA e Rússia: {athletes_from_usa_russia}")


Países dos atletas: ['USA', 'Canada', 'China', 'Mexico', 'Russia']
Nomes de atletas com 'J': ['John Doe', 'Jane Smith']
Atletas dos EUA e Rússia: ['John Doe', 'Olga Ivanova']



### Por que usar seletores CSS?
Os seletores CSS permitem uma maneira rápida e eficiente de **identificar e extrair elementos** de um arquivo HTML, com base em atributos como classes (`.class`), IDs (`#id`), ou a hierarquia dos elementos. São amplamente utilizados porque são simples e intuitivos, especialmente se você já trabalha com desenvolvimento web ou conhece CSS básico.

### Casos indicados para o uso de seletores CSS:
- **Páginas HTML bem estruturadas**: Quando os elementos HTML estão organizados de forma consistente, com uso claro de classes e IDs.
- **Extração específica**: Ideal para buscar dados com base em atributos, como todos os elementos que possuem uma classe ou ID específica.
- **Scraping simples e direto**: Quando a estrutura da página é estável e você pode confiar na consistência do HTML ao longo do tempo.

### Casos em que seletores CSS não são recomendados:
- **Páginas dinâmicas**: Quando o conteúdo da página é carregado via **JavaScript** (como em **Single Page Applications**), os seletores CSS podem não capturar o conteúdo, já que ele não estará presente no HTML inicial.
- **Estruturas HTML inconsistentes**: Quando o HTML não segue um padrão claro de classes ou IDs, tornando difícil a extração precisa de dados.
- **Manipulação de dados complexos**: Se a estrutura do documento é mais complexa, como no caso de documentos XML rigorosos, o **XPath** pode ser mais eficiente e preciso para navegar e extrair informações.

### Conclusão
Seletores CSS são uma ótima ferramenta para **web scraping** quando se trabalha com páginas HTML bem organizadas e estáticas. No entanto, para páginas dinâmicas ou estruturas mais complexas, o uso de ferramentas mais robustas, como **Selenium** para páginas JavaScript ou **XPath** para navegação avançada, pode ser mais apropriado.


# Exercício 4:

Utilize seletores de CSS com BeautifulSoup e algum tratamento em Python que seja necessário para alocar as informações de cada atleta do arquivo HTML em um dataframe do Pandas a ser salvo em arquivo CSV e então responda:

A média de idades dos atletas;
O atleta ganhador do Speed Test;
O atleta ganhador do Endurance Test.

In [13]:

from bs4 import BeautifulSoup  # Usada para parsing do HTML
import pandas as pd  

# Abrindo o arquivo HTML e carregando seu conteúdo com BeautifulSoup
with open("C:\infnet_ultimo_semestre\TP3_web_scraping\TP3.html", "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")  # Carrega o conteúdo do HTML em um objeto BeautifulSoup

# Criamos uma lista para armazenar as informações de cada atleta
athletes = []

# Selecionamos todos os elementos <div> que possuem a classe "athlete" (ou seja, cada bloco de informações de um atleta)
athlete_divs = soup.select(".athlete")

# Iteramos sobre cada bloco de informações de atleta
for div in athlete_divs:
    # Extraímos o nome do atleta, contido na tag <h2> com a classe "name"
    name = div.select_one(".name").text
    
    # Extraímos o país do atleta, contido na tag <p> com a classe "country"
    country = div.select_one(".country").text
    
    # Extraímos a idade do atleta, contida na tag <p> com a classe "age" e convertemos para um inteiro
    age = int(div.select_one(".age").text)
    
    # Extraímos o resultado do Speed Test, contido na primeira tag <span> com a classe "result" (primeira ocorrência)
    speed_test = int(div.select(".test .result")[0].text)  # O primeiro teste listado é o de velocidade
    
    # Extraímos o resultado do Endurance Test, contido na segunda tag <span> com a classe "result" (segunda ocorrência)
    endurance_test = int(div.select(".test .result")[1].text)  # O segundo teste listado é o de resistência
    
    # Adicionamos todas essas informações em um dicionário e armazenamos na lista "athletes"
    athletes.append({
        "Name": name,
        "Country": country,
        "Age": age,
        "Speed Test": speed_test,
        "Endurance Test": endurance_test
    })

# Criamos um DataFrame do Pandas com a lista de atletas (cada dicionário vira uma linha)
df = pd.DataFrame(athletes)

# Salvamos o DataFrame em um arquivo CSV para persistir os dados
df.to_csv("athletes_data.csv", index=False)  # O parâmetro index=False garante que não seja salva uma coluna de índice

# Agora, realizamos as operações solicitadas para responder às perguntas:

# 1. Calculamos a média de idades dos atletas
average_age = df["Age"].mean()  # Usamos a função mean() do Pandas para calcular a média de idades

# 2. Encontramos o atleta com o maior resultado no Speed Test
# A função idxmax() retorna o índice da linha com o valor máximo na coluna "Speed Test"
# Usamos esse índice para pegar o nome do atleta ganhador
winner_speed_test = df.loc[df["Speed Test"].idxmax(), "Name"]

# 3. Encontramos o atleta com o maior resultado no Endurance Test
# Usamos a mesma lógica do Speed Test para encontrar o ganhador do Endurance Test
winner_endurance_test = df.loc[df["Endurance Test"].idxmax(), "Name"]

# Exibindo os resultados:
print(f"Média de idades dos atletas: {average_age:.2f}")  # Mostra a média com duas casas decimais
print(f"Atleta ganhador do Speed Test: {winner_speed_test}")
print(f"Atleta ganhador do Endurance Test: {winner_endurance_test}")


Média de idades dos atletas: 26.00
Atleta ganhador do Speed Test: John Doe
Atleta ganhador do Endurance Test: Jane Smith


# Explicação sobre o Uso de BeautifulSoup para Extração de Dados

**BeautifulSoup** é uma biblioteca Python usada para **parsing** de documentos HTML e XML, facilitando a navegação e extração de dados de páginas web. No contexto deste exercício, ela foi usada para capturar informações de atletas como nome, país, idade e resultados de testes a partir de um arquivo HTML.

### Vantagens:
- **Simplicidade**: Oferece uma sintaxe intuitiva para navegar em documentos HTML usando seletores CSS.
- **Flexibilidade**: Pode lidar com HTML malformado, sendo mais permissiva que outras ferramentas de parsing.
- **Integração com seletores CSS**: Permite selecionar elementos diretamente por classes e IDs, similar ao CSS usado em páginas web.

### Exemplos práticos:
- **`.select_one(".name")`**: Seleciona o elemento que contém o nome do atleta com a classe `name`.
- **`.select(".test .result")`**: Captura os resultados dos testes de velocidade e resistência, com base nas classes `test` e `result`.
- **`.select(".athlete")`**: Seleciona todos os blocos de informações de cada atleta, que possuem a classe `athlete`.

### Integração com Pandas:
Após extrair os dados com **BeautifulSoup**, eles são organizados em um **DataFrame do Pandas** para manipulação e análise, permitindo cálculos como a **média de idades** e a identificação dos atletas vencedores dos testes de velocidade e resistência.

Essa técnica é ideal para **extração e estruturação de dados de páginas HTML**, facilitando o processamento e análise posterior com ferramentas como o Pandas.


# Parte 3 WebScraping 

## Exercício 5: 

Acesse a URL https://www.scrapethissite.com/pages/simple/ e obtenha os dados de todos os países, salvando em um dataframe (sem utilizar o comando `pd.read_html`) com as seguintes colunas:

Country;
Capital;
Population;
Area.

# Estrutura do Site

O site contém informações sobre países organizadas em blocos de `div` com a classe `.country`. Cada bloco apresenta os seguintes elementos:

- **Nome do país**: Dentro de uma tag `<h3>` com a classe `.country-name`.
- **Capital**: Dentro de um `span` com a classe `.country-capital`.
- **População**: Dentro de um `span` com a classe `.country-population`.
- **Área**: Dentro de um `span` com a classe `.country-area`.

Esses blocos estão agrupados dentro de várias `divs` com a classe `.col-md-4 country`. Isso permite que os dados sejam extraídos de maneira consistente, com base nas classes CSS específicas para cada informação.


In [14]:
import requests
from bs4 import BeautifulSoup


# URL da página que vamos acessar
url = "https://www.scrapethissite.com/pages/simple/"

# Realizando a requisição GET para a página
response = requests.get(url)

# Verificamos se a requisição foi bem-sucedida (status code 200)
if response.status_code == 200:
    # Parsing do conteúdo da página HTML com BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Criamos uma lista para armazenar as informações de cada país
    countries_data = []
    
    # Selecionamos todas as divs que contém informações dos países (classe "country")
    country_divs = soup.select(".country")
    
    # Iteramos sobre cada div de país e extraímos as informações relevantes
    for country_div in country_divs:
        # Nome do país
        country_name = country_div.select_one(".country-name").text.strip()
        
        # Capital do país
        capital = country_div.select_one(".country-capital").text.strip()
        
        # População do país
        population = country_div.select_one(".country-population").text.strip()
        
        # Área do país
        area = country_div.select_one(".country-area").text.strip()
        
        # Adicionamos os dados a uma lista como dicionário
        countries_data.append({
            "Country": country_name,
            "Capital": capital,
            "Population": population,
            "Area": area
        })
    
    # Criamos um DataFrame com os dados dos países
    df = pd.DataFrame(countries_data)
    
    # Salvamos o DataFrame em um arquivo CSV
    df.to_csv("countries_data.csv", index=False)
    
    # Exibindo o DataFrame
    print(df)
else:
    print(f"Erro ao acessar a página. Status code: {response.status_code}")


                  Country           Capital Population       Area
0                 Andorra  Andorra la Vella      84000      468.0
1    United Arab Emirates         Abu Dhabi    4975593    82880.0
2             Afghanistan             Kabul   29121286   647500.0
3     Antigua and Barbuda        St. John's      86754      443.0
4                Anguilla        The Valley      13254      102.0
..                    ...               ...        ...        ...
245                 Yemen             Sanaa   23495361   527970.0
246               Mayotte         Mamoudzou     159042      374.0
247          South Africa          Pretoria   49000000  1219912.0
248                Zambia            Lusaka   13460305   752614.0
249              Zimbabwe            Harare   11651858   390580.0

[250 rows x 4 columns]


# Explicação sobre o uso do BeautifulSoup no exercício acima

**BeautifulSoup** é utilizado para fazer o **parsing** do HTML da página e permitir a navegação e extração dos dados. Neste caso específico, usamos seletores CSS para localizar os elementos desejados:

- **`.select(".country")`**: Seleciona todos os blocos de países.
- **`.select_one(".country-name")`**: Extrai o nome do país.
- **`.select_one(".country-capital")`**: Extrai a capital do país.
- **`.select_one(".country-population")`**: Extrai a população do país.
- **`.select_one(".country-area")`**: Extrai a área do país.

Após a extração dos dados, eles são organizados em uma lista de dicionários, que é convertida em um **DataFrame do Pandas** para fácil manipulação e análise.


# Exercício 6: 

Escolha uma notícia qualquer do site https://www.romanews.com.br/ e obtenha o HTML da página via requisição utilizando a biblioteca que preferir (`urllib`, `requests`, etc.), salvando o arquivo no seu diretório. Então, a partir do arquivo HTML da notícia salva, construa códigos em Python utilizando `BeautifulSoup` para obter e salvar em um arquivo JSON, junto à URL da notícia e ao datetime do momento da requisição da página (os objetos datetime devem ter a informação da timezone e ser colocados em isoformat no momento de gerar o JSON):

O objeto datetime da data e hora da publicação da notícia (dica: é sempre bom checar as tags `meta` dentro do `head` do arquivo.);
O título da notícia;
O corpo do texto da notícia;
O subtítulo da notícia (se houver);
O autor ou autores da notícia (se houver).

In [15]:
pip install -q selenium


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import json
from datetime import datetime
import pytz
import time
import os

# URL da notícia que queremos capturar
url = "https://www.romanews.com.br/cidades/pesquisas-da-uepa-ajudam-na-fiscalizacao-de-transporte-e-comercializacao-da-madeira-no-para-0924"

# Caminho completo para o ChromeDriver (certifique-se de que o caminho está correto)
chromedriver_path = r"C:\Program Files\chromedriver-win64\chromedriver.exe"

# Inicializamos o WebDriver do Selenium com a classe Service
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

# Acessamos a URL da notícia utilizando o WebDriver do Selenium
driver.get(url)

# Pausamos a execução por alguns segundos para garantir que a página tenha tempo suficiente para carregar todo o conteúdo
time.sleep(5)

# Capturamos o HTML da página carregada pelo navegador controlado pelo Selenium
page_content = driver.page_source

# Fechamos o WebDriver para liberar o navegador e evitar que ele continue aberto
driver.quit()

# Salvamos o HTML capturado em um arquivo local para referência futura
html_file_path = "noticia_romanews.html"
with open(html_file_path, "w", encoding="utf-8") as file:
    file.write(page_content)

# Parsing do conteúdo HTML capturado usando BeautifulSoup
soup = BeautifulSoup(page_content, "html.parser")

# Obter o datetime da requisição com timezone
timezone = pytz.timezone('America/Sao_Paulo')
request_datetime = datetime.now(timezone).isoformat()

# Capturando a data de publicação do meta tag
pub_datetime_tag = soup.find("meta", {"name": "cXenseParse:recs:publishtime"})
pub_datetime = pub_datetime_tag["content"] if pub_datetime_tag else "Data de publicação não encontrada"

# Capturando o título da notícia a partir da tag <title>
title_tag = soup.find("title")
title = title_tag.text.strip() if title_tag else "Título não encontrado"

# Extraindo o corpo da notícia (isso pode precisar de ajuste dependendo do HTML completo)
body_tag = soup.find("div", class_="corpo")  # Ajuste necessário para capturar o corpo real
body = body_tag.get_text(separator=" ").strip() if body_tag else "Corpo da notícia não encontrado"

# Criar um dicionário com os dados extraídos da notícia
news_data = {
    "url": url,
    "request_datetime": request_datetime,
    "publication_datetime": pub_datetime,
    "title": title,
    "body": body
}

# Verificar se a pasta 'resultados' existe, senão, criá-la
if not os.path.exists('resultados'):
    os.makedirs('resultados')

# Salvamos o dicionário em um arquivo JSON dentro da pasta 'resultados'
json_file_path = os.path.join('resultados', "noticia_selenium.json")
with open(json_file_path, "w", encoding="utf-8") as json_file:
    json.dump(news_data, json_file, ensure_ascii=False, indent=4)

print(f"Notícia salva com sucesso no arquivo {json_file_path}")


Notícia salva com sucesso no arquivo resultados\noticia_selenium.json


## O que é o Selenium?
Selenium é uma biblioteca usada para automatizar navegadores da web, simulando ações como navegar para uma URL, clicar em botões, preencher formulários e, no nosso caso, capturar o conteúdo dinâmico de uma página.

## Quando utilizar Selenium?
Utilizamos Selenium quando o conteúdo da página é gerado dinamicamente via JavaScript, ou seja, quando a página carrega elementos após a inicialização (algo comum em sites modernos). Outras bibliotecas como `requests` ou `urllib` não conseguem capturar esses conteúdos dinâmicos. O Selenium abre um navegador real (ou sem interface gráfica) para interagir com o conteúdo exatamente como um usuário faria.

### Vantagens do Selenium:
- **Captura de conteúdo dinâmico**: Ideal para sites que usam JavaScript para carregar ou modificar o conteúdo.
- **Simulação de ações do usuário**: Podemos navegar, clicar, preencher formulários, e muito mais.
- **Flexibilidade**: Funciona em diversos navegadores (Chrome, Firefox, etc.) e oferece controle completo da página.

### Desvantagens:
- **Mais lento que requests/urllib**: Por controlar um navegador real, a execução do Selenium é mais lenta.
- **Dependência de WebDriver**: Precisamos configurar corretamente o WebDriver (como o ChromeDriver no nosso caso).

## Contexto do código:
1. **Selenium é utilizado** para carregar dinamicamente a página de uma notícia da web (Romanews) e capturar o HTML completo.
2. **BeautifulSoup** é então utilizado para fazer o parsing desse HTML e extrair dados específicos, como o título, autor, e data de publicação.
3. **Json e os**: Os dados extraídos são salvos em um arquivo JSON, garantindo que podemos reutilizar essas informações posteriormente.
4. **time e datetime**: Garantimos que a página tenha tempo para carregar com `time.sleep()`, e usamos `datetime` para registrar o momento exato da requisição, incluindo o fuso horário com `pytz`.

### Por que Selenium foi necessário aqui?
O site Romanews possivelmente carrega partes do conteúdo de forma dinâmica (via JavaScript), e com Selenium conseguimos garantir que o navegador tenha carregado todo o conteúdo antes de extraí-lo.


# Parte 4 WebCrawling

## Exercício 8: 

Examine o site https://difusoranews.com/ e descubra o padrão de URL para paginação que ele aceita (dica: considere tentativa-e-erro). Então, utilize-o para obter uma lista de links de notícias requisitando as cinco primeiras páginas e raspando os links através de um único seletor de CSS aplicado via `BeautifulSoup`.

In [17]:
import requests
from bs4 import BeautifulSoup

# Padrões de URL para testar
pagination_patterns = [
    "https://difusoranews.com/page/{}",       # Padrão 1
    "https://difusoranews.com/?pagina={}",    # Padrão 2
    "https://difusoranews.com/p/{}",          # Padrão 3
    "https://difusoranews.com/?pg={}",        # Padrão 4
    "https://difusoranews.com/index/page/{}", # Padrão 5
    "https://difusoranews.com/index?page={}", # Padrão 6
    "https://difusoranews.com/list/page/{}",  # Padrão 7
    "https://difusoranews.com/?step={}",      # Padrão 8
]

# Função para verificar se a página retornada é diferente da página inicial
def is_valid_pagination(soup_initial, soup_page):
    # Tentamos comparar algo único, como o título da página ou o número de links
    return soup_initial != soup_page

# Obter o conteúdo da página inicial
response_initial = requests.get("https://difusoranews.com")
soup_initial = BeautifulSoup(response_initial.content, "html.parser")

# Loop pelos padrões de URL
for pattern in pagination_patterns:
    for page_num in range(1, 6):  # Testando até a 5ª página
        url = pattern.format(page_num)
        print(f"Tentando URL: {url}")
        response = requests.get(url)
        
        if response.status_code == 200:
            soup_page = BeautifulSoup(response.content, "html.parser")
            
            # Verifica se a página é diferente da inicial
            if is_valid_pagination(soup_initial, soup_page):
                print(f"Página {page_num} encontrada com o padrão: {url}")
            else:
                print(f"A página {page_num} com o padrão {url} parece ser igual à página inicial.")
        else:
            print(f"Erro ao tentar acessar {url}: Status {response.status_code}")

print("Teste de padrões de URL concluído.")



Tentando URL: https://difusoranews.com/page/1
A página 1 com o padrão https://difusoranews.com/page/1 parece ser igual à página inicial.
Tentando URL: https://difusoranews.com/page/2
Página 2 encontrada com o padrão: https://difusoranews.com/page/2
Tentando URL: https://difusoranews.com/page/3
Página 3 encontrada com o padrão: https://difusoranews.com/page/3
Tentando URL: https://difusoranews.com/page/4
Página 4 encontrada com o padrão: https://difusoranews.com/page/4
Tentando URL: https://difusoranews.com/page/5
Página 5 encontrada com o padrão: https://difusoranews.com/page/5
Tentando URL: https://difusoranews.com/?pagina=1
Página 1 encontrada com o padrão: https://difusoranews.com/?pagina=1
Tentando URL: https://difusoranews.com/?pagina=2
Página 2 encontrada com o padrão: https://difusoranews.com/?pagina=2
Tentando URL: https://difusoranews.com/?pagina=3
Página 3 encontrada com o padrão: https://difusoranews.com/?pagina=3
Tentando URL: https://difusoranews.com/?pagina=4
Página 4 enc

In [18]:
import requests
from bs4 import BeautifulSoup

# URL base e padrão de paginação identificado
base_url = "https://difusoranews.com/page/{}/"

# Lista para armazenar os links das notícias
news_links = []

# Função para extrair links das notícias de uma página
def extract_news_links(soup):
    # Usando o seletor CSS que identifica os links das notícias
    # Aqui estamos assumindo que os links das notícias estão em <a> tags dentro de <h2> (ou outro padrão que você identificar no site)
    for article in soup.select('h2 a'):
        news_links.append(article['href'])

# Loop para percorrer as 5 primeiras páginas
for page_num in range(1, 6):
    url = base_url.format(page_num)
    print(f"Acessando: {url}")
    
    # Requisição HTTP para obter o conteúdo da página
    response = requests.get(url)
    
    if response.status_code == 200:
        # Parsing do conteúdo da página com BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Extração dos links de notícias dessa página
        extract_news_links(soup)
        
        print(f"Links extraídos da página {page_num}: {len(news_links)} até agora")
    else:
        print(f"Erro ao acessar a página {page_num}. Status code: {response.status_code}")

# Exibindo os links extraídos
print("Lista final de links de notícias extraídos:")
for link in news_links:
    print(link)

# Opcional: Salvando os links em um arquivo de texto
with open('news_links.txt', 'w') as file:
    for link in news_links:
        file.write(link + "\n")

print("Links de notícias salvos em 'news_links.txt'.")


Acessando: https://difusoranews.com/page/1/
Links extraídos da página 1: 0 até agora
Acessando: https://difusoranews.com/page/2/
Links extraídos da página 2: 0 até agora
Acessando: https://difusoranews.com/page/3/
Links extraídos da página 3: 0 até agora
Acessando: https://difusoranews.com/page/4/
Links extraídos da página 4: 0 até agora
Acessando: https://difusoranews.com/page/5/
Links extraídos da página 5: 0 até agora
Lista final de links de notícias extraídos:
Links de notícias salvos em 'news_links.txt'.


In [19]:
import requests
from bs4 import BeautifulSoup

# Padrão de paginação identificado
pagination_pattern = "https://difusoranews.com/page/{}"

# Função para extrair links de notícias
def extract_news_links(soup):
    news_links = []
    # Seleciona todos os elementos <a> dentro da tag <main>, que podem conter as notícias
    for link in soup.select("main a[href]"):
        href = link.get("href")
        if href and href.startswith("https://difusoranews.com/") and href not in news_links:
            news_links.append(href)
    return news_links

# Lista para armazenar todos os links de notícias
all_news_links = []

# Loop pelas 5 primeiras páginas
for page_num in range(1, 6):
    url = pagination_pattern.format(page_num)
    print(f"Acessando URL: {url}")
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        # Extrair os links de notícias da página atual
        news_links = extract_news_links(soup)
        if news_links:
            print(f"Links encontrados na página {page_num}: {news_links}")
            all_news_links.extend(news_links)  # Adicionar os links à lista total
        else:
            print(f"Nenhum link encontrado na página {page_num}.")
    else:
        print(f"Erro ao acessar a página {url}: Status {response.status_code}")

# Salvando os links em um arquivo txt
with open("news_link.txt", "w") as f:
    for link in all_news_links:
        f.write(link + "\n")

print("Links de notícias extraídos e salvos em 'news_link.txt'.")



Acessando URL: https://difusoranews.com/page/1
Links encontrados na página 1: ['https://difusoranews.com/eleicoes-2024/sabatina/saulo-arcangeli-defende-reducao-de-salario-e-criacao-de-empresa-de-onibus-da-prefeitura/', 'https://difusoranews.com/eleicoes-2024/sabatina/', 'https://difusoranews.com/eleicoes-2024/', 'https://difusoranews.com/eleicoes-2024/pesquisa-mostra-josinaldo-moraes-liderando-corrida-eleitoral-em-santa-helena/', 'https://difusoranews.com/oportunidade/', 'https://difusoranews.com/oportunidade/sextou-no-painel-de-emprego-com-vaga-para-fisioterapeuta/', 'https://difusoranews.com/eleicoes-2024/candidatos-a-prefeitura-de-sao-luis-divulgam-agenda-desta-sexta-feira-13/', 'https://difusoranews.com/esportes/rayssa-leal-garante-vaga-para-semifinal-do-mundial-de-skate-street-em-roma/', 'https://difusoranews.com/esportes/', 'https://difusoranews.com/eleicoes-2024/jhulio-sousa-lidera-disputa-pela-prefeitura-de-pocao-de-pedras-diz-pesquisa/', 'https://difusoranews.com/eleicoes-2024



## Bibliotecas Utilizadas
1. **Requests**: 
   - Utilizamos a biblioteca `requests` para fazer as requisições HTTP e acessar as páginas do site. Essa biblioteca é simples e eficiente para recuperar o conteúdo HTML de qualquer página web.
   
2. **BeautifulSoup (bs4)**: 
   - Usamos o `BeautifulSoup` para realizar o parsing do HTML recuperado e facilitar a extração de dados das páginas. A partir dessa ferramenta, conseguimos utilizar seletores CSS e outras técnicas para localizar elementos do DOM, como links de notícias.

## Estratégia de Paginação
- O site **Difusora News** não possuía um padrão claro de paginação. Portanto, fizemos um processo de tentativa e erro para descobrir o padrão correto. Após testes, confirmamos que o padrão correto para acessar páginas adicionais de notícias era o seguinte:
   - `https://difusoranews.com/page/{numero_da_pagina}`

- Esse padrão permitiu que acessássemos e raspássemos dados de várias páginas, de forma iterativa, usando um laço para cobrir as cinco primeiras páginas.

## Função de Verificação
- Desenvolvemos uma função para comparar o HTML da página inicial com as páginas subsequentes. O objetivo era garantir que as páginas raspadas fossem de fato diferentes da página inicial, evitando duplicações e resultados inválidos.

## Armazenamento de Links
- Após coletar os links de cada página, salvamos esses links em um arquivo `news_links.txt`. Esse processo é fundamental para estruturar e organizar os dados extraídos de forma a permitir futuros processamentos e análises.

## Aplicação Prática
Essa estratégia de **web scraping** é valiosa para automação de coleta de dados em larga escala. Por exemplo, no contexto de monitoramento de notícias (como o nosso caso), a empresa poderia automatizar o acompanhamento de notícias específicas, extraindo informações relevantes (títulos, links, textos) para análise de mídia, sentimento ou tendências de mercado.


**Exercício 9:**

Repita a questão anterior utilizando um site de notícias de sua escolh

In [20]:
import requests
from bs4 import BeautifulSoup

# URL do site de notícias escolhido (CNN Brasil)
base_url = "https://www.cnnbrasil.com.br/"

# Padrões de paginação para teste
pagination_patterns = [
    "https://www.cnnbrasil.com.br/page/{}",       # Padrão 1
    "https://www.cnnbrasil.com.br/?pagina={}",    # Padrão 2
    "https://www.cnnbrasil.com.br/noticias/page/{}", # Padrão 3
    "https://www.cnnbrasil.com.br/?pg={}",        # Padrão 4
]

# Função para verificar se a página atual é diferente da inicial
def is_valid_pagination(soup_initial, soup_page):
    return soup_initial != soup_page

# Obter conteúdo da página inicial
response_initial = requests.get(base_url)
soup_initial = BeautifulSoup(response_initial.content, "html.parser")

# Lista para armazenar os links de notícias
news_links = []

# Loop para testar padrões de paginação
for pattern in pagination_patterns:
    for page_num in range(1, 6):  # Testando as 5 primeiras páginas
        url = pattern.format(page_num)
        print(f"Acessando URL: {url}")
        response = requests.get(url)

        if response.status_code == 200:
            soup_page = BeautifulSoup(response.content, "html.parser")

            # Verifica se a página atual é diferente da inicial
            if is_valid_pagination(soup_initial, soup_page):
                # Extração dos links de notícias
                for link in soup_page.select("a[href]"):  # Seleciona todos os links <a>
                    href = link.get("href")
                    if href and "cnnbrasil.com.br" in href:
                        news_links.append(href)
                print(f"Links encontrados na página {page_num}.")
            else:
                print(f"A página {page_num} parece ser igual à página inicial.")
        else:
            print(f"Erro ao acessar {url}: Status {response.status_code}")

# Salvando os links em um arquivo .txt
with open("cnn_news_links.txt", "w") as f:
    for link in news_links:
        f.write(f"{link}\n")

print("Links de notícias extraídos e salvos em 'cnn_news_links.txt'.")


Acessando URL: https://www.cnnbrasil.com.br/page/1
Erro ao acessar https://www.cnnbrasil.com.br/page/1: Status 404
Acessando URL: https://www.cnnbrasil.com.br/page/2
Erro ao acessar https://www.cnnbrasil.com.br/page/2: Status 404
Acessando URL: https://www.cnnbrasil.com.br/page/3
Erro ao acessar https://www.cnnbrasil.com.br/page/3: Status 404
Acessando URL: https://www.cnnbrasil.com.br/page/4
Erro ao acessar https://www.cnnbrasil.com.br/page/4: Status 404
Acessando URL: https://www.cnnbrasil.com.br/page/5
Erro ao acessar https://www.cnnbrasil.com.br/page/5: Status 404
Acessando URL: https://www.cnnbrasil.com.br/?pagina=1
Links encontrados na página 1.
Acessando URL: https://www.cnnbrasil.com.br/?pagina=2
Links encontrados na página 2.
Acessando URL: https://www.cnnbrasil.com.br/?pagina=3
Links encontrados na página 3.
Acessando URL: https://www.cnnbrasil.com.br/?pagina=4
Links encontrados na página 4.
Acessando URL: https://www.cnnbrasil.com.br/?pagina=5
Links encontrados na página 5.


## Exercício 10: 

Consulte o arquivo `robots.txt` do site https://tecnoblog.net/ para encontrar o link do `sitemap_index` e obtenha a partir dele a lista de todos os sitemaps de notícias (filtrando apenas os de notícias).

In [23]:
import requests
from bs4 import BeautifulSoup

# Definir o URL do sitemap
sitemap_url = "https://tecnoblog.net/sitemap_index.xml"

# Headers para simular um navegador
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Fazer a requisição para o sitemap_index.xml
response = requests.get(sitemap_url, headers=headers)

if response.status_code == 200:
    # Parser do conteúdo XML
    soup = BeautifulSoup(response.content, 'xml')
    
    # Extrair todos os sitemaps
    sitemaps = soup.find_all('sitemap')
    
    # Filtrar e imprimir apenas os sitemaps de notícias
    for sitemap in sitemaps:
        loc = sitemap.find('loc').text
        if "noticia" in loc or "news" in loc:
            print(f"Sitemap de notícias encontrado: {loc}")
            
            # Fazer requisição para cada sitemap de notícias
            news_sitemap_response = requests.get(loc, headers=headers)
            if news_sitemap_response.status_code == 200:
                news_soup = BeautifulSoup(news_sitemap_response.content, 'xml')
                news_links = news_soup.find_all('url')
                
                # Extrair e mostrar os links das notícias
                for news in news_links:
                    news_url = news.find('loc').text
                    print(f"Notícia encontrada: {news_url}")
else:
    print(f"Erro ao acessar o sitemap. Status code: {response.status_code}")


Sitemap de notícias encontrado: https://tecnoblog.net/noticias-sitemap.xml
Notícia encontrada: https://tecnoblog.net/noticias/
Notícia encontrada: https://tecnoblog.net/noticias/mercado-livre-vai-mudar-taxa-de-frete-gratis-e-subsidio-para-vendedores/
Notícia encontrada: https://tecnoblog.net/noticias/nova-operadora-devera-levar-sinal-em-4g-para-36-mil-km-de-estradas/
Notícia encontrada: https://tecnoblog.net/noticias/game-of-thrones-e-friends-dominam-retrospectiva-2021-da-hbo-max-no-brasil/
Notícia encontrada: https://tecnoblog.net/noticias/celular-simples-da-samsung-galaxy-a03-core-e-homologado-pela-anatel/
Notícia encontrada: https://tecnoblog.net/noticias/pubg-new-state-e-lancado-para-android-e-iphone-mas-trava-celulares/
Notícia encontrada: https://tecnoblog.net/noticias/loggi-monta-megagalpao-em-sp-para-processar-ate-1-milhao-de-pacotes-por-dia/
Notícia encontrada: https://tecnoblog.net/noticias/smartphone-e-campeao-de-queixas-sobre-blackfriday-no-reclame-aqui/
Notícia encontrada:

# Resumo do Exercício - Consultando o Sitemap do Tecnoblog

## O que fizemos:

1. **Acessamos o arquivo `robots.txt`** do site `https://tecnoblog.net/` para identificar o **sitemap_index.xml**.
   - O arquivo `robots.txt` nos ajuda a descobrir quais URLs são permitidos para scraping, além de indicar o link do sitemap.
   
2. **Realizamos uma requisição HTTP** para acessar o `sitemap_index.xml` usando a biblioteca `requests`.
   - Para simular um navegador e evitar bloqueios, incluímos um **header de User-Agent** para parecer que estamos acessando o site de um navegador real.

3. **Filtramos os sitemaps relevantes**, ou seja, aqueles que contêm notícias. 
   - Para isso, usamos o `BeautifulSoup` para analisar o XML retornado e identificamos URLs que contêm "noticia" ou "news" na sua estrutura.

4. **Extraímos os links das páginas de notícias**:
   - Para cada sitemap de notícias encontrado, fizemos outra requisição e extraímos todos os links das notícias disponíveis no sitemap.

## Por que simulamos um navegador?

Simular o acesso de um navegador real por meio de um **User-Agent** é importante porque muitos sites restringem acessos feitos por bots. Sem essa simulação, a requisição poderia ser bloqueada ou retornaria uma resposta inadequada, como vimos no erro 403 que enfrentamos antes de ajustar o código.

Essa abordagem de simulação é bastante comum em web scraping, pois ajuda a evitar restrições impostas por mecanismos de segurança do site.

## Lógica por trás:

1. **Consulta ao `robots.txt`**: 
   - Identificamos se o site permite a indexação e scraping. O arquivo também nos dá a rota para o **sitemap**, que é um índice de todas as URLs do site organizadas de forma estruturada.

2. **Requisições para o Sitemap**:
   - Uma vez que temos o link do `sitemap_index.xml`, podemos explorar todas as páginas de notícias de maneira eficiente.
   - Filtramos apenas os sitemaps que contêm notícias, garantindo que os dados extraídos sejam relevantes ao nosso objetivo.

3. **Scraping Estruturado**:
   - Usar um sitemap é uma abordagem eficiente porque ele já organiza as URLs do site, permitindo que o scraping seja feito de maneira sistemática e menos propensa a erros.
   
## Uso Comercial:

Essa abordagem pode ser utilizada em **estratégias de inteligência competitiva**, onde uma empresa coleta dados de notícias de concorrentes ou do setor para:
- **Monitorar tendências de mercado**.
- **Acompanhar notícias relevantes para tomada de decisão**.
- **Análise de reputação**, observando como seu nome ou produtos estão sendo mencionados na mídia.

Além disso, empresas podem usar essa técnica para **agregação de conteúdo**, automatizando a coleta de links de notícias para gerar insights, relatórios ou alimentar outros sistemas.



# Parte 5 Scrapy

## Exercício 11: 

Escolha um dos sites de exemplo em https://toscrape.com/ para criar um projeto no Scrapy que obtenha os principais dados disponíveis, salvando em CSV ou JSON conforme a conveniência em relação aos dados do site escolhido.

In [24]:
pip install -q scrapy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Scrapy: Introdução, Passo a Passo e Aplicação Comercial

Scrapy é um framework de web scraping e crawling desenvolvido em Python, projetado para extrair grandes volumes de dados de websites. Ele se destaca por sua eficiência, pois faz múltiplas requisições simultâneas, coletando dados de forma rápida e escalável. Ao contrário de ferramentas como Selenium, que simula a interação do usuário em um navegador, o Scrapy foca diretamente na obtenção de informações estruturadas (HTML) e trabalha em segundo plano, sem a necessidade de renderizar as páginas inteiras, tornando-o mais leve e performático.

### Passo a Passo

1. **Instalação do Scrapy**: O primeiro passo foi instalar o Scrapy utilizando o comando `pip install scrapy` no terminal. Essa instalação configura o ambiente para criar e rodar projetos de scraping.
   
2. **Criação do Projeto**: Utilizamos o comando `scrapy startproject bookscraper` para iniciar um novo projeto chamado `bookscraper`. Esse comando gera a estrutura necessária para o desenvolvimento, com pastas como `spiders`, onde serão criados os scripts que definem os robôs (spiders).

3. **Criação do Spider**: Para começar o scraping, criamos um spider que coleta dados de um site específico, no caso, o site **Books to Scrape**. Isso foi feito com o comando `scrapy genspider books toscrape.com`, gerando um arquivo onde configuramos os detalhes do scraping, como as URLs iniciais, os domínios permitidos e a lógica para extrair os dados.

4. **Execução e Salvamento dos Dados**: Após configurar o spider, executamos o scraping com `scrapy crawl books`, e o Scrapy percorre todas as páginas definidas, extraindo os dados de interesse, como títulos, preços e disponibilidade dos livros. Os dados podem ser salvos em formatos como JSON ou CSV, usando o comando `scrapy crawl books -o books.json` para exportar as informações coletadas para um arquivo.

### Aplicação Comercial

O uso comercial do Scrapy é vasto. Um exemplo prático seria a construção de sistemas de monitoramento de preços para e-commerces, onde um spider coleta diariamente os preços de produtos dos concorrentes e gera relatórios que permitem a empresa ajustar suas estratégias de precificação de forma dinâmica. Essa automação proporciona insights valiosos com baixo custo de operação.


## Exercício 12: 

Escolha um dos sites utilizados nas questões 6, 7, 8 e 9 para montar um projeto no Scrapy que abarque tanto o Crawling quanto o Scraping, a fim de rodá-lo tal como na questão anterior.

## Para este exercício, vamos utilizamos o site CNN do exercício 9. Os resultados foram salvos em json no diretório do projeto
